In [3]:
import tensorflow as tf
import os
import pandas as pd
from tqdm import tqdm


In [4]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
##data
train_data_root = "/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data"
data_ds = tf.data.experimental.make_csv_dataset(
            os.path.join(train_data_root, "train-dataset.csv"),
            batch_size=64,
            label_name='label',
            # na_value="0",
            num_epochs=1,
            shuffle=True,
            ignore_errors=True)
val_ds = data_ds.take(1000)
train_ds = data_ds.skip(1000)



2023-01-15 17:01:10.158554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-15 17:01:10.668482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22310 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:a1:00.0, compute capability: 8.6


In [6]:
data_path = '/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data'
data = pd.read_csv(os.path.join(data_path,'train-dataset.csv'))

In [7]:
data.head()

,sampleId,label,pvId,suv,itemId,userSeq,logTs,operator,browserType,deviceType,osType,province,city
0,1,0,1420332726637uyIZR2O,1581173072343wnwm3q,10092752,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
1,2,0,1420332726637uyIZR2O,1581173072343wnwm3q,10085565,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
2,3,0,1420332726637uyIZR2O,1581173072343wnwm3q,10105937,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
3,4,0,1420332726637uyIZR2O,1581173072343wnwm3q,10082274,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140
4,5,0,1420332726637uyIZR2O,1581173072343wnwm3q,10083446,10094375:1640918979673;10084835:1640836934361;...,1641039890894,2203920,1583865,2326891,3585570,1518261,3058140


In [ ]:
#数据处理


In [8]:
##config
EMBEDDING_SIZE = 10
HIDDEN_SIZE = 128

# 1.制作inputs

In [9]:
inputs = {
    'itemId' : tf.keras.layers.Input(name='itemId', shape=(), dtype='int32'),
    'city' : tf.keras.layers.Input(name='city', shape=(), dtype='int32')
}


In [10]:
inputs

{'itemId': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'itemId')>,
 'city': <KerasTensor: shape=(None,) dtype=int32 (created by layer 'city')>}

# 2.制作feature_columns

In [11]:
feature_columns = {}
feature_columns['itemId'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_identity(key='itemId',num_buckets=100000) ,EMBEDDING_SIZE)
feature_columns['city'] = tf.feature_column.embedding_column(tf.feature_column.categorical_column_with_identity(key='city',num_buckets=4164526) ,EMBEDDING_SIZE)

In [12]:
feature_columns

{'itemId': EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='itemId', number_buckets=100000, default_value=None), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f407474ff70>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True),
 'city': EmbeddingColumn(categorical_column=IdentityCategoricalColumn(key='city', number_buckets=4164526, default_value=None), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f407474ffd0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True)}

In [13]:
item_features = tf.keras.layers.DenseFeatures(feature_columns['city'])(inputs)

In [14]:
#model
outputs = tf.keras.layers.Dense(1024,activation='relu')(item_features)
outputs = tf.keras.layers.Dense(256,activation='relu')(outputs)
outputs = tf.keras.layers.Dense(1,activation='sigmoid')(outputs)
model = tf.keras.Model(inputs, outputs)

In [15]:

model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'])

#train
history = model.fit(train_ds, epochs=1)

/root/miniconda3/lib/python3.8/site-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['sampleId', 'pvId', 'suv', 'userSeq', 'logTs', 'operator', 'browserType', 'deviceType', 'osType', 'province'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


     25/Unknown - 3s 7ms/step - loss: 0.6688 - accuracy: 0.7819

2023-01-15 17:01:30.331856: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


65154/65154 [==============================] - 286s 4ms/step - loss: 0.4896 - accuracy: 0.8075


In [16]:
test_loss, test_accuracy = model.evaluate(val_ds)


1000/1000 [==============================] - 4s 4ms/step - loss: 0.5206 - accuracy: 0.7875
